# Task 2: Recommendation Engine - Skeleton Notebook

### Setting up the Notebook

In [1]:
import numpy as np
import pandas as pd

from src.utils import read_csv
from src.data_preprocessor import DataPreprocessor

from src.recommendation_utils import top_k_cosine_distance, top_k_nearest_neighbors

In [2]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### Load the Data

In [3]:
trainX, trainY = read_csv('data/train.csv', ylabel='price')

auxSubzone, _ = read_csv('data/auxiliary-data/sg-subzones.csv')

auxInfraDict = {}
Infralist = ['sg-commerical-centres', 'sg-mrt-stations', 'sg-primary-schools', 'sg-secondary-schools', 'sg-shopping-malls']
for ele in Infralist:
    auxInfra, _ = read_csv('data/auxiliary-data/' + ele + '.csv')
    auxInfraDict[ele] = auxInfra

data_preprocessor = DataPreprocessor(auxSubzone, auxInfraDict)
trainX, trainY = data_preprocessor.fit_transform(trainX, trainY)

df = pd.concat([trainX, trainY], axis=1)

## Computing the Top Recommendations

In [12]:
def get_top_recommendations(row, df, k=3):
    # index_list = top_k_cosine_distance(row, df, k=k)
    # index_list = top_k_nearest_neighbors(row, df, k=k)
        
    return index_list

## Testing the Recommendation Engine

### Pick a Sample Listing as Input

In [13]:
# Pick a row id of choice
row_id = 10
#row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row = df.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
data_preprocessor.inverse_transform(pd.DataFrame([row]))

,address,property_name,property_type,built_year,num_beds,num_baths,size_sqft,total_num_units,lat,lng,...,nearest_sg-primary-schools,density_sg-primary-schools,nearest_sg-secondary-schools,density_sg-secondary-schools,nearest_sg-shopping-malls,density_sg-shopping-malls,subzone_area_size,price,furnishing,floor_level
10,299 bedok south avenue 3,bedok court,condo,1985.0,2.0,4.0,1733.0,280.0,1.322153,103.945223,...,0.004778,3.368502,0.003085,3.972056,0.016131,2.012918,2.9973,2205000.0,unspecified,high


### Compute and Display the recommendations

In [15]:
k = 3

recommendation_list = get_top_recommendations(pd.DataFrame([row]), df, k=k)

data_preprocessor.inverse_transform(df.iloc[recommendation_list])

,address,property_name,property_type,built_year,num_beds,num_baths,size_sqft,total_num_units,lat,lng,...,nearest_sg-primary-schools,density_sg-primary-schools,nearest_sg-secondary-schools,density_sg-secondary-schools,nearest_sg-shopping-malls,density_sg-shopping-malls,subzone_area_size,price,furnishing,floor_level
10,299 bedok south avenue 3,bedok court,condo,1985.0,2.0,4.0,1733.0,280.0,1.322153,103.945223,...,0.004778,3.368502,0.003085,3.972056,0.016131,2.012918,2.9973,2205000.0,unspecified,high
3160,295 bedok south avenue 3,bedok court,condo,1985.0,2.0,2.0,1733.0,280.0,1.322153,103.945223,...,0.004778,3.368502,0.003085,3.972056,0.016131,2.012918,2.9973,2205000.0,unfurnished,high
12352,295 bedok south avenue 3,bedok court,condo,1985.0,2.0,2.0,1733.0,280.0,1.322153,103.945223,...,0.004778,3.368502,0.003085,3.972056,0.016131,2.012918,2.9973,2205000.0,unspecified,mid
